In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
import time
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


In [2]:
df = pd.read_csv('train.csv')
df.head(10)

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,payment_delay
0,HI,33,area_code_415,no,no,0,200.5,117,34.09,159.9,111,13.59,196.2,84,8.83,16.3,6,4.40,3,no
1,TN,80,area_code_415,yes,no,0,276.5,122,47.01,195.6,79,16.63,210.3,78,9.46,7.2,3,1.94,1,yes
2,CT,37,area_code_408,no,no,0,134.9,98,22.93,248.4,130,21.11,236.2,113,10.63,14.7,2,3.97,3,no
3,TN,106,area_code_415,no,no,0,119.2,142,20.26,228.4,139,19.41,197.9,61,8.91,8.4,9,2.27,2,no
4,TX,123,area_code_408,no,no,0,260.9,85,44.35,168.5,103,14.32,178.3,91,8.02,13.3,5,3.59,3,no
5,CT,152,area_code_408,no,yes,20,239.1,105,40.65,209.1,111,17.77,268.2,130,12.07,13.3,3,3.59,5,no
6,NY,87,area_code_415,no,no,0,204.8,101,34.82,161.0,80,13.69,285.7,89,12.86,9.5,3,2.57,0,no
7,UT,110,area_code_415,no,no,0,271.1,108,46.09,237.0,122,20.15,239.9,122,10.80,9.8,5,2.65,2,yes
8,IL,66,area_code_415,no,yes,21,134.4,110,22.85,136.2,104,11.58,215.6,105,9.70,9.7,4,2.62,3,no
9,MI,119,area_code_510,yes,yes,22,172.1,119,29.26,223.6,133,19.01,150.0,94,6.75,13.9,20,3.75,1,yes


In [3]:
#Creating labels 

labels = df['payment_delay']
labels = labels.replace(('yes', 'no'), (1,0))
labels = np.array(labels)

**EDA**

In [4]:
# no. of training examples

print(df.shape)

(3000, 20)


In [5]:
df.isnull().sum()

state                            0
account_length                   0
area_code                        0
international_plan               0
voice_mail_plan                  0
number_vmail_messages            0
total_day_minutes                0
total_day_calls                  0
total_day_charge                 0
total_eve_minutes                0
total_eve_calls                  0
total_eve_charge                 0
total_night_minutes              0
total_night_calls                0
total_night_charge               0
total_intl_minutes               0
total_intl_calls                 0
total_intl_charge                0
number_customer_service_calls    0
payment_delay                    0
dtype: int64

In [6]:
# Counting the NaN values so we eliminate them.

df.isna().sum()
# df.dropna()

state                            0
account_length                   0
area_code                        0
international_plan               0
voice_mail_plan                  0
number_vmail_messages            0
total_day_minutes                0
total_day_calls                  0
total_day_charge                 0
total_eve_minutes                0
total_eve_calls                  0
total_eve_charge                 0
total_night_minutes              0
total_night_calls                0
total_night_charge               0
total_intl_minutes               0
total_intl_calls                 0
total_intl_charge                0
number_customer_service_calls    0
payment_delay                    0
dtype: int64

In [7]:
#Drop any duplicated rows

df = df.drop_duplicates()
print(df.shape)

(3000, 20)


Seems like there were no NaN, null values or duplicate rows. Great, the dataset is clean! </br> </br> 
Since we're interested in predicting the payment delays, we'll split the dataset in **2 parts**: one part contains information about the customers that registered payment delays and the other part containts information about those who didn't. We'll only analise the statistic parameters of each column: **mean** and **standard deviation**, so we can get an idea of the mathematical distribution that features these values.

In [8]:
df_yes = df[df['payment_delay'] == 'yes'].describe(percentiles=[])
print('No. of customers who delayed their payment: ', df[df['payment_delay'] == 'yes'].shape[0])
df_yes = df_yes[1:3]
df_yes

No. of customers who delayed their payment:  413


,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
mean,103.765133,4.622276,209.580872,100.104116,35.629201,211.148426,99.687651,17.947579,206.382082,99.581114,9.287264,10.683051,4.285714,2.884964,2.256659
std,38.769615,11.377362,67.943348,20.021501,11.550457,49.828265,19.755711,4.235185,51.128992,19.604124,2.300929,2.859689,2.720065,0.771956,1.849163


In [9]:
df_no = df[df['payment_delay'] == 'no'].describe(percentiles=[])
print('No. of customers who paid on time: ', df[df['payment_delay'] == 'no'].shape[0])
df_no = df_no[1:3]
df_no

No. of customers who paid on time:  2587


,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
mean,99.990336,8.512949,175.806997,100.248550,29.887762,198.259644,100.336683,16.852319,200.256320,99.996908,9.011631,10.130421,4.528798,2.735717,1.485504
std,39.581515,13.996993,50.014753,19.726153,8.502478,49.956122,19.764935,4.246245,50.780369,19.683300,2.285141,2.724591,2.449064,0.735540,1.173339


In [10]:
data = [go.Bar(x=df_yes.columns, y=df_yes.loc['mean'], name='Mean - Payment delay'),
        go.Bar(x=df_yes.columns, y=df_no.loc['mean'], name='Mean - Payment on time')]

layout = go.Layout(barmode='group')

fig = dict(data = data, layout = layout)
iplot(fig, show_link=False)

data = [go.Bar(x=df_yes.columns, y=df_yes.loc['std'], name='Standard dev - Payment delay'),
    go.Bar(x=df_yes.columns, y=df_no.loc['std'], name='Standard dev - Payment on time')]

layout = go.Layout(barmode='group')

fig = dict(data = data, layout = layout)
iplot(fig, show_link=False)

It looks like most of the customers that registered payment delay spend more time on the phone than the other ones, especially during daytime. Regarding these customers, in most cases, the mean and standard deviation of the common columns is higher, which results in higher values.

We'll further analyze the "binary" columns, the one that only contain yes/no values. 

In [11]:
# y.head(20)
fig = px.pie(df[df['payment_delay'] == 'yes'], names='international_plan', title='Payment Delay - International Plan')
fig.show()

fig = px.pie(df[df['payment_delay'] == 'no'], names='international_plan', title='Payment on time - International Plan')
fig.show()


It seems like most of the customers that did not register payment delays didn't have a international plan subscription. However, the majority of those who did not pay on time didn't subscribe to this plan as well, so we can say that this feature is impure. 

In [12]:
# y.head(20)
fig = px.pie(df[df['payment_delay'] == 'yes'], names='voice_mail_plan', title='Payment Delay - Voice Mail Plan')
fig.show()

fig = px.pie(df[df['payment_delay'] == 'no'], names='voice_mail_plan', title='Payment on time - Voice Mail Plan')
fig.show()


Again, we can't decide on the payment delay probability, based on this feature.

In [13]:
fig = px.pie(df[df['payment_delay'] == 'yes'], names='state', title='Payment Delay - State')
fig.show()

fig = px.pie(df[df['payment_delay'] == 'no'], names='state', title='Payment on time - State')
fig.show()

Seems like there are no outliers in the state distribution of the customers.

In [14]:
fig = px.pie(df[df['payment_delay'] == 'yes'], names='area_code', title='Payment Delay - Area code')
fig.show()

fig = px.pie(df[df['payment_delay'] == 'no'], names='area_code', title='Payment on time - Area code')
fig.show()

No difference in the area code distribution.

In order to predict the payment delay probability, I firstly used the **random forest machine learning method**. For preprocessing the data, we need to encode the string data using the **one hot encoding technique** in order to allow the machine learning model to process it. 

In [15]:
df = df.drop(['payment_delay'], axis = 1)

In [16]:
df_one_hot = pd.get_dummies(df)
print(df_one_hot.shape)

(3000, 73)


In [17]:
df_one_hot.head()

,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,...,state_WI,state_WV,state_WY,area_code_area_code_408,area_code_area_code_415,area_code_area_code_510,international_plan_no,international_plan_yes,voice_mail_plan_no,voice_mail_plan_yes
0,33,0,200.5,117,34.09,159.9,111,13.59,196.2,84,...,0,0,0,0,1,0,1,0,1,0
1,80,0,276.5,122,47.01,195.6,79,16.63,210.3,78,...,0,0,0,0,1,0,0,1,1,0
2,37,0,134.9,98,22.93,248.4,130,21.11,236.2,113,...,0,0,0,1,0,0,1,0,1,0
3,106,0,119.2,142,20.26,228.4,139,19.41,197.9,61,...,0,0,0,0,1,0,1,0,1,0
4,123,0,260.9,85,44.35,168.5,103,14.32,178.3,91,...,0,0,0,1,0,0,1,0,1,0


In [18]:
# def standard_scale(dataframe):
#     for i in df_yes.columns:
#         dataframe[i] = (dataframe[i] - dataframe[i].mean())/dataframe[i].std()
#     return dataframe

In [19]:
# df_one_hot = standard_scale(df_one_hot)
df_one_hot.head()

,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,...,state_WI,state_WV,state_WY,area_code_area_code_408,area_code_area_code_415,area_code_area_code_510,international_plan_no,international_plan_yes,voice_mail_plan_no,voice_mail_plan_yes
0,33,0,200.5,117,34.09,159.9,111,13.59,196.2,84,...,0,0,0,0,1,0,1,0,1,0
1,80,0,276.5,122,47.01,195.6,79,16.63,210.3,78,...,0,0,0,0,1,0,0,1,1,0
2,37,0,134.9,98,22.93,248.4,130,21.11,236.2,113,...,0,0,0,1,0,0,1,0,1,0
3,106,0,119.2,142,20.26,228.4,139,19.41,197.9,61,...,0,0,0,0,1,0,1,0,1,0
4,123,0,260.9,85,44.35,168.5,103,14.32,178.3,91,...,0,0,0,1,0,0,1,0,1,0


In [20]:
dataset = df_one_hot.to_numpy()

First, I will split the training dataset into a new training set and a test set. Thus, I will check the model performance on a test data with known labels, in order to determine its prediction accuracy.

In [21]:
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(dataset, labels, test_size = 0.25, random_state = 30)


In [22]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (2250, 73)
Training Labels Shape: (2250,)
Testing Features Shape: (750, 73)
Testing Labels Shape: (750,)


In [23]:
# Initialize model with N decision trees
def train_model(N, train_features, train_labels):
    model = RandomForestRegressor(n_estimators = N, random_state = 42)
    
    # Train the model on training data
    tic = time.time()
    model.fit(train_features, train_labels);
    tac = time.time()
    print('Time taken: ', np.round(tac-tic, 3),' seconds.')
    return model

def test_model(model, test_features):
    predictions = model.predict(test_features)
    predictions = np.where(predictions > 0.5, 1, 0)
    print((predictions==1).sum())
    # print((predictions==test_labels).sum())
    print('Accuracy = ', np.round((predictions==test_labels).sum()/len(test_labels)*100, 2), '%')

The model predicts labels with values in [0,1]. However, I'll consider that any value below 0.5 is considered 0 (no), and the rest of them are considered 1 (yes).

In [24]:
# 1st case: N = 100(default trees number)
model = train_model(100, train_features, train_labels)
test_model(model, test_features)

Time taken:  2.04  seconds.
87
Accuracy =  94.0 %


The accuracy is great, but it can do better...I'll initialize the random forest with N = 200 trees

In [25]:
#2nd case: N = 200
model = train_model(200, train_features, train_labels)
test_model(model, test_features)

Time taken:  3.681  seconds.
88
Accuracy =  94.67 %


Let's see how it handels the test dataset for N = 400 trees.

In [26]:
#2nd case: N = 400
model = train_model(400, train_features, train_labels)
test_model(model, test_features)

Time taken:  7.315  seconds.
87
Accuracy =  94.53 %


I think this is the best the model can do. </br> </br> I will now train the model on the proposed test set.

In [27]:
test = pd.read_csv('test.csv')
test.head()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls
0,AK,36,area_code_408,no,yes,30,146.3,128,24.87,162.5,80,13.81,129.3,109,5.82,14.5,6,3.92,0
1,AK,104,area_code_408,no,no,0,278.4,106,47.33,81.0,113,6.89,163.2,137,7.34,9.8,5,2.65,1
2,AK,126,area_code_415,no,no,0,58.2,94,9.89,138.7,118,11.79,136.8,91,6.16,11.9,1,3.21,5
3,AK,130,area_code_415,no,no,0,242.5,101,41.23,102.8,114,8.74,142.4,89,6.41,9.3,2,2.51,2
4,AK,78,area_code_510,no,no,0,190.3,88,32.35,194.5,89,16.53,256.5,109,11.54,11.7,5,3.16,2


In [28]:
# test = test.drop('state', axis = 1)
test_one_hot = pd.get_dummies(test)
# test_one_hot = standard_scale(test_one_hot)
new_test_features = test_one_hot.to_numpy()
print(new_test_features.shape)

(2000, 73)


I'll train the model using the whole provided training dataset.

In [29]:
model = train_model(200, dataset, labels)
predictions = model.predict(new_test_features)

Time taken:  4.99  seconds.


In [30]:
a = np.sort(predictions)[::-1]

In [31]:
print((a[:300]>0.5).sum())

238


Considering my assumption (values less than 0.5 are equivalent to 'no', the other ones - 'yes'), looks like only 238 customers out of 2000 will delay their payment. However, since the task is to export the first 300 customers who are likely to register such delays, I'll just concatenate the predictions array to the initial test dataset and sort the values accordingly.

In [32]:
test.insert(test.shape[1], 'payment_delay_prob[%]', predictions*100)

In [33]:
test = test.sort_values('payment_delay_prob[%]', ascending = False)

In [34]:
test.head(300)

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,payment_delay_prob[%]
1014,MT,64,area_code_415,no,no,0,131.3,105,22.32,109.4,114,9.30,246.2,125,11.08,12.7,4,3.43,4,100.0
708,MA,119,area_code_408,yes,yes,16,147.2,103,25.02,160.1,96,13.61,184.0,120,8.28,7.7,2,2.08,0,100.0
1075,ND,147,area_code_408,yes,yes,35,157.5,109,26.78,189.6,67,16.12,227.0,76,10.22,11.1,2,3.00,3,100.0
72,AL,91,area_code_415,no,no,0,271.9,106,46.22,231.9,116,19.71,247.0,110,11.12,9.3,3,2.51,1,100.0
1835,WA,100,area_code_408,no,no,0,70.8,94,12.04,215.6,102,18.33,230.8,125,10.39,9.5,1,2.57,6,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,SC,84,area_code_415,no,no,0,274.1,119,46.60,144.1,123,12.25,268.2,100,12.07,9.9,5,2.67,2,27.5
772,MD,156,area_code_510,no,no,0,81.7,133,13.89,242.5,117,20.61,203.8,112,9.17,12.3,6,3.32,3,27.5
1681,TX,138,area_code_415,no,no,0,217.2,73,36.92,190.8,127,16.22,133.5,97,6.01,5.2,3,1.40,4,27.0
667,LA,149,area_code_415,no,yes,20,198.9,77,33.81,274.0,88,23.29,190.7,76,8.58,14.3,9,3.86,1,27.0
